In [ ]:
!pip install keras-tuner

     |████████████████████████████████| 96 kB 6.1 MB/s 
  Created wheel for kt-legacy: filename=kt_legacy-1.0.3-py3-none-any.whl size=9568 sha256=3e2c5d11c7dbc76c1202dc2c3646e094254fb83a0f347919d7a6a39d1d545a34
  Stored in directory: /root/.cache/pip/wheels/38/5c/e0/13003e68c17f403af40b92a24d20171b95fef13b0fdaba833c
Successfully built kt-legacy


In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [ ]:
fashion_mnist=keras.datasets.fashion_mnist

In [ ]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [ ]:
train_images=train_images/255.0
test_images=test_images/255.0

In [ ]:
train_images[0].shape

(28, 28)

In [ ]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [ ]:
def build_model(hp):
    # create model object
    model = keras.Sequential([
    #adding first convolutional layer    
    keras.layers.Conv2D(
        #adding filter 
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        # adding filter size or kernel size
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        #activation function
        activation='relu',
        input_shape=(28,28,1)),
    # adding second convolutional layer 
    keras.layers.Conv2D(
        #adding filter 
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        #adding filter size or kernel size
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        #activation function
        activation='relu'
    ),
    # adding flatten layer    
    keras.layers.Flatten(),
    # adding dense layer    
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    # output layer    
    keras.layers.Dense(10, activation='softmax')
    ])
    #compilation of model
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
    return model

In [ ]:
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  """Entry point for launching an IPython kernel.


In [ ]:
tunersearch=RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory='output',project_name='Mnist_Fashion')

In [ ]:
tunersearch.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 3 Complete [00h 29m 22s]
val_accuracy: 0.8636666536331177

Best val_accuracy So Far: 0.9123333096504211
Total elapsed time: 00h 54m 07s

Search: Running Trial #4

Hyperparameter    |Value             |Best Value So Far 
conv_1_filter     |48                |48                
conv_1_kernel     |5                 |5                 
conv_2_filter     |48                |48                
conv_2_kernel     |3                 |3                 
dense_1_units     |80                |128               
learning_rate     |0.001             |0.001             

Epoch 1/3
1688/1688 [==============================] - 140s 83ms/step - loss: 0.4003 - accuracy: 0.8561 - val_loss: 0.3018 - val_accuracy: 0.8910
Epoch 2/3
1688/1688 [==============================] - 140s 83ms/step - loss: 0.2630 - accuracy: 0.9028 - val_loss: 0.2823 - val_accuracy: 0.8943
Epoch 3/3
 893/1688 [==============>...............] - ETA: 1:03 - loss: 0.2064 - accuracy: 0.9216

In [ ]:
model=tunersearch.get_best_models(num_models=1)[0]

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 112)       2912      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20, 20, 64)        179264    
_________________________________________________________________
flatten (Flatten)            (None, 25600)             0         
_________________________________________________________________
dense (Dense)                (None, 80)                2048080   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                810       
Total params: 2,231,066
Trainable params: 2,231,066
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(train_images,train_labels,epochs=10,validation_split=0.1,initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 10s 6ms/step - loss: 0.1710 - accuracy: 0.9357 - val_loss: 0.2690 - val_accuracy: 0.9098
Epoch 5/10
1688/1688 [==============================] - 9s 6ms/step - loss: 0.1320 - accuracy: 0.9500 - val_loss: 0.2835 - val_accuracy: 0.9123
Epoch 6/10
1688/1688 [==============================] - 9s 6ms/step - loss: 0.1025 - accuracy: 0.9614 - val_loss: 0.3138 - val_accuracy: 0.9143
Epoch 7/10
1688/1688 [==============================] - 9s 6ms/step - loss: 0.0811 - accuracy: 0.9698 - val_loss: 0.3019 - val_accuracy: 0.9167
Epoch 8/10
1688/1688 [==============================] - 9s 6ms/step - loss: 0.0587 - accuracy: 0.9787 - val_loss: 0.4003 - val_accuracy: 0.9105
Epoch 9/10
1688/1688 [==============================] - 10s 6ms/step - loss: 0.0513 - accuracy: 0.9806 - val_loss: 0.4358 - val_accuracy: 0.9117
Epoch 10/10
1688/1688 [==============================] - 10s 6ms/step - loss: 0.0441 - accuracy: 0.9846 - val_loss: 0.4350 - val_accur